<a href="https://colab.research.google.com/github/shinichiromizuno/QueryMultiTopic/blob/master/Multi_BERTSum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount your Google Drive.
# Connect to GPU in Google Colab.

In [ ]:
################ Preprocess ################

In [ ]:
# Configure working directories.　We use the preprocessed dataset output of BERT-Base by configuring it as the source directory.
work_dir =  '/content/drive/MyDrive/work_Multi-BERTSum'
source_dir = '/content/drive/MyDrive/work_BERT-Base'

In [ ]:
# Install required packages and configure module path.
!pip install transformers
!pip install pyrouge
%env MODULE_PATH=/content/drive/MyDrive/QueryMultiTopic

In [ ]:
# Skipping Step1~3, as we use the preprocessed dataset output of BERT-Base.

In [ ]:
# Step4. Creating Labeled JSON Files
import json
import torch
from transformers import BertTokenizer
import os
import shutil
from glob import glob

dir = work_dir + '/json_data_labeled'
if os.path.exists(dir):
  shutil.rmtree(dir)
os.mkdir(dir)

# stride = float('inf')
stride = 5

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
for data in sorted(glob(source_dir + '/json_data/*')):
  json_open = open(data)
  json_load = json.load(json_open)
  keys = list(json_load.keys())
  keys.remove('src')
  src = json_load['src']
  ds_dict = {}
  
  # Adding labels by category
  for i in range(1,17+1):
    labels = [0] * len(src)
    tgt_idx = 'tgt' + str(i)
    if tgt_idx not in keys:
      ds_dict[tgt_idx] = labels
      continue
    tgts = json_load[tgt_idx]
    for tgt in tgts:
      lent = len(tgt)
      tcon = []
      for j in tgt[0:lent]:
          tcon += j
      tjoin = ' '.join(tcon)
      last = len(src) - lent
      for k in range(0, last+1):
        scon = []
        for m in src[k:k+lent]:
          scon += m
        sjoin = ' '.join(scon)
        if sjoin == tjoin:
          for l in range(k, k+lent):
            labels[l] = 1
    ds_dict[tgt_idx] = labels
  
  # Adding CLS and SEP, and tokenize to ids
  src_subtoken_idxs_list = []
  src_txt = []
  for sent in src:
    text = ' '.join(sent)
    src_txt.append(text)
    src_subtokens = tokenizer.tokenize('[CLS] ' + text + ' [SEP]')
    src_subtoken_idxs = tokenizer.convert_tokens_to_ids(src_subtokens)
    src_subtoken_idxs_list.append(src_subtoken_idxs)

  # Adding starting positions(stpos) and number of sentences to be included(nstbi)
  for i in range(1,17+1):
    stpos, nstbi, ipos, scnt, nums, tflag = [], [], 0, 0, 0, False
    max_scnt = 512
    while ipos < len(src_subtoken_idxs_list) and tflag is False:
      nums, scnt = 0, 0
      if len(src_subtoken_idxs_list[ipos+nums]) > max_scnt:
        src_subtoken_idxs_list[ipos+nums] = src_subtoken_idxs_list[ipos+nums][:max_scnt]
      # Ending process if it exceeds 512 tokens or processes last token
      while scnt <= max_scnt:
        # Ending with Terminate flag if it processes last token
        if ipos+nums > len(src_subtoken_idxs_list) - 1:
          tflag = True
          nums += 1
          break
        scnt += len(src_subtoken_idxs_list[ipos+nums])
        nums += 1
      nums -= 1
      stpos.append(ipos)
      nstbi.append(nums)
      ipos += stride if nums > stride else nums
    ds_dict['stpos'+str(i)] = stpos
    ds_dict['nstbi'+str(i)] = nstbi

  ds_dict['src_ids'] = src_subtoken_idxs_list
  ds_dict['src_txt'] = src_txt
  
  filename = data.split('/')[-1]
  with open(dir + '/Proposed.' + filename, mode='w') as f:
    f.write(json.dumps(ds_dict))

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Check Statistics
from glob import glob
import json

cnt_dict = {'src':0}
for i in range(1,17+1):
  cnt_dict['tgt' + str(i)] = 0

for data in sorted(glob(source_dir + '/json_data_labeled/*')):
  json_open = open(data)
  json_load = json.load(json_open)
  keys = list(json_load.keys())
  cnt_dict['src'] += len(json_load['src_txt'])
  for i in range(1,17+1):
    tgt_idx = 'tgt' + str(i)
    cnt_dict[tgt_idx] += sum(json_load[tgt_idx])
print(cnt_dict)

{'src': 173664, 'tgt1': 1493, 'tgt2': 1338, 'tgt3': 8891, 'tgt4': 3932, 'tgt5': 6201, 'tgt6': 2849, 'tgt7': 6938, 'tgt8': 10217, 'tgt9': 8102, 'tgt10': 4522, 'tgt11': 6078, 'tgt12': 9676, 'tgt13': 8761, 'tgt14': 2985, 'tgt15': 4482, 'tgt16': 3815, 'tgt17': 6630}


In [ ]:
# Step5. Creating PT Files
import json
import itertools
import random
import os
import shutil
from glob import glob
import torch
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

dir = work_dir + '/pt_data'
if os.path.exists(dir):
  shutil.rmtree(dir)
os.mkdir(dir)

sep_vid = tokenizer.vocab['[SEP]']
cls_vid = tokenizer.vocab['[CLS]']
pad_vid = tokenizer.vocab['[PAD]']

for data in sorted(glob(work_dir + '/json_data_labeled/*')):
  json_open = open(data)
  json_load = json.load(json_open)
  src_ids = json_load['src_ids']
  for idx in range(1, 18):
    sub_dir = dir + '/' + str(idx)
    if not os.path.exists(sub_dir):
      os.mkdir(sub_dir)

    dataset = {'src': json_load['src_txt'], 'tgts': {}}
    datasets = []
    tgt = 'tgt' + str(idx)
    dataset['tgts'][str(idx)] = json_load[tgt]
    for stpos, nstbi in zip(json_load['stpos'+str(idx)], json_load['nstbi'+str(idx)]):
      pt_dict = {}
      pt_dict['qry'] = str(idx)
      # Adding start position and end position
      pt_dict['stpos'] = stpos
      enpos = stpos+nstbi
      pt_dict['enpos'] = enpos
      # Adding source ids and source text
      src_subtoken_idxs = list(itertools.chain.from_iterable(src_ids[stpos:enpos]))
      pt_dict['src_ids'] = src_subtoken_idxs
      # Adding Segment ID
      _segs = [-1] + [i for i, t in enumerate(src_subtoken_idxs) if t == sep_vid]
      segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
      segments_ids = []
      for i, s in enumerate(segs):
        if (i % 2 == 0):
          segments_ids += s * [0]
        else:
          segments_ids += s * [1]
      pt_dict['segs'] = segments_ids
      # Adding CLS positions
      cls_ids = [i for i, t in enumerate(src_subtoken_idxs) if t == cls_vid]
      pt_dict['clss'] = cls_ids
      # Adding labels
      pt_dict['tgt'] = json_load[tgt][stpos:enpos]
      datasets.append(pt_dict)
    dataset['ds'] = datasets
    filename = '.'.join(data.split('/')[-1].split('.')[1:5])
    torch.save(dataset, sub_dir + '/' + filename + '.pt')

In [ ]:
# Check Number of Dataset by Query
from glob import glob
import torch
for i in range(1,17+1):
  ds_counts = []
  zero_sent_cnts = []
  z_cnt = 0
  o_cnt = 0
  for data_type in ['train', 'test', 'valid']:
    ds_list = glob(work_dir + '/pt_data/' + str(i) + '/' + data_type + '.*')
    ds_count = 0
    for ds in ds_list:
      dataset = torch.load(ds)
      ds_count += len(dataset['ds'])
      for d in dataset['ds']:
        z_cnt += len([j for j in d['tgt'] if j == 0])
        o_cnt += len([j for j in d['tgt'] if j == 1])
    ds_counts.append(ds_count)
  print(f'qry: {i}, train: {ds_counts[0]}, test: {ds_counts[1]}, valid: {ds_counts[2]}, ds_total: {ds_counts[0] + ds_counts[1] + ds_counts[2]}, zero_sent_in_ds: {z_cnt}, one_sent_in_ds: {o_cnt}')

qry: 1, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 515066, one_sent_in_ds: 4466
qry: 2, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 515661, one_sent_in_ds: 3871
qry: 3, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 492415, one_sent_in_ds: 27117
qry: 4, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 507526, one_sent_in_ds: 12006
qry: 5, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 500503, one_sent_in_ds: 19029
qry: 6, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 510655, one_sent_in_ds: 8877
qry: 7, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 498311, one_sent_in_ds: 21221
qry: 8, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 488591, one_sent_in_ds: 30941
qry: 9, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 495034, one_sent_in_ds: 24498
qry:

In [ ]:
# Check Output
from glob import glob
import torch
dataset = torch.load(glob(work_dir + '/pt_data/12/*.1.Ahresty.2020.pt')[0])
print(dataset['src'])
print(dataset['tgts'])
print(len(dataset['ds']))
for d in dataset['ds']:
  print(d)
  break

['We aim to create an affluent society by pursuing each of these areas and integrating them .', 'Our corporate name , Ahresty , comes from “ RST . ”', 'It was created by linking together the pronunciation of the first letters in Research , Service , and Technology .', 'Research means continuous research , investigation , and development of new technologies , new markets , and new sales techniques .', 'Service means providing warm , attentive service through personal interaction .', 'Technology means truly excellent technology that incorporates both physical and soft aspects and is highly beneficial for society .', 'These three areas of Research , Service , and Technology can not be considered independently .', 'Both technology and a spirit of service are necessary to accomplish the research involved in R&D .', 'To explain it in another way , research , service , and technology are intricately linked and each supports the others .', 'It is an organic relationship in which each component

In [ ]:
# Preparing Data for Validation & Test
import json
import random
from glob import glob
import torch

src_doc_valid = []
tgt_list_valid = []
src_doc_test = []
tgt_list_test = []

for data in sorted(glob(work_dir + '/json_data_labeled/*')):
  filetype = data.split('/')[-1].split('.')[1]
  json_open = open(data)
  json_load = json.load(json_open)
  src_txt = json_load['src_txt']
  if filetype == 'valid':
    src_doc_valid.append(src_txt)
    tgt_list = []
    for i in range(1,18):
      tgt_list.append(json_load['tgt'+str(i)])
    tgt_list_valid.append(tgt_list)
  if filetype == 'test':
    src_doc_test.append(src_txt)
    tgt_list = []
    for i in range(1,18):
      tgt_list.append(json_load['tgt'+str(i)])
    tgt_list_test.append(tgt_list)

# Incorporate SDGs goal text in query text
sdgsdir = '/content/drive/MyDrive/DatasetESG/0-SDGs/'

all_query_txt = []
for goal in sorted(glob(sdgsdir + '/*')):
  query_txt = []
  with open(goal, mode='r') as f:
    for line in f:
      query_txt.append(line.strip())
      break
  all_query_txt.append(' '.join(query_txt))

dataset = {'src_doc_valid': src_doc_valid, 'tgt_list_valid': tgt_list_valid, 'src_doc_test': src_doc_test, 'tgt_list_test': tgt_list_test, 'all_query_txt': all_query_txt}
torch.save(dataset, work_dir + '/dataset.pt')  

In [ ]:
################ Training ################

In [ ]:
# Install Pytorch
!pip3 install torchtext==0.3.1
!pip3 install torch==1.6.0
!pip install tensorboardX

     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0
     |████████████████████████████████| 748.8 MB 16 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.


     |████████████████████████████████| 125 kB 14.4 MB/s 


In [ ]:
# Training Models by Query
!rm -r $work_dir/models
!mkdir $work_dir/models
!mkdir logs

for idx in range(1, 18):
  %env BERT_DATA_PATH=$work_dir/pt_data/$idx/ 
  %env MODEL_PATH=$work_dir/models/proposed$idx
  %env LOG_PATH=/content/logs/proposed$idx
  !python $MODULE_PATH/src/train.py -mode train -summarizer proposed -dropout 0.1 -bert_data_path $BERT_DATA_PATH -model_path $MODEL_PATH -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 -world_size 1 -report_every 10 -save_checkpoint_steps 2000 -batch_size 4000 -decay_method noam -train_steps 10000 -accum_count 2 -log_file $LOG_PATH -use_interval true -warmup_steps 10000 -ff_size 2048 -inter_layers 2 -heads 8

env: BERT_DATA_PATH=/content/drive/MyDrive/work_proposed/pt_data/1/
env: MODEL_PATH=/content/drive/MyDrive/work_proposed/models/proposed1
env: LOG_PATH=/content/logs/proposed1
[2022-03-29 07:00:48,608 INFO] Device ID 0
[2022-03-29 07:00:48,608 INFO] Device cuda
proposed!
Downloading: 100% 570/570 [00:00<00:00, 567kB/s]
Downloading: 100% 420M/420M [00:06<00:00, 69.0MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This 

In [ ]:
################ Validation ################

In [ ]:
# Validation
!cp $MODULE_PATH/bert_config_uncased_base_proposed.json ../bert_config_uncased_base.json
!rm -r $work_dir/results
!mkdir $work_dir/results
!rm -r $work_dir/logs
!mkdir $work_dir/logs
%env LOG_PATH=$work_dir/logs/proposed

for idx in range(1, 18):
  !mkdir $work_dir/results/$idx
  %env RESULT_PATH=$work_dir/results/$idx/ 
  %env BERT_DATA_PATH=$work_dir/pt_data/$idx/
  %env MODEL_PATH=$work_dir/models/proposed$idx
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_21000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_22000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_23000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposed -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_24000.pt -block_trigram true

env: LOG_PATH=/content/drive/MyDrive/work_proposed/logs/proposed
env: RESULT_PATH=/content/drive/MyDrive/work_proposed/results/1/
env: BERT_DATA_PATH=/content/drive/MyDrive/work_proposed/pt_data/1/
env: MODEL_PATH=/content/drive/MyDrive/work_proposed/models/proposed1
[2022-04-23 21:13:57,998 INFO] Loading checkpoint from /content/drive/MyDrive/work_proposed/models/proposed1/model_step_21000.pt
Namespace(accum_count=1, batch_size=512, bert_config_path='../bert_config_uncased_base.json', bert_data_path='/content/drive/MyDrive/work_proposed/pt_data/1/', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, ff_size=2048, gpu_ranks=[0], heads=8, hidden_size=128, inter_layers=2, log_file='/content/drive/MyDrive/work_proposed/logs/proposed', lr=1, max_grad_norm=0, mode='valid', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='/content/drive/MyDrive/work_proposed/resul

In [ ]:
# Exploring Optimal Hyper-Parameters
def predict_goals(cos_sim_list, threshold):
  src_pred_list = []
  for cos_sim in cos_sim_list:
    ones = torch.ones(cos_sim.shape).int()
    zeros = torch.zeros(cos_sim.shape).int()
    pred = torch.where(cos_sim > threshold, ones, zeros)
    src_pred_list.append(pred.tolist())
  return src_pred_list

In [ ]:
def gen_optim_list(pt_optim_dicts_all, qry, priority):
  optim_dicts_all = []
  for pt_optim_dicts in pt_optim_dicts_all:
    optim_dicts = {}
    optim_dicts[qry] = {}
    for pt_optim_dict in pt_optim_dicts[qry]:
      stpos = pt_optim_dict['stpos']
      enpos = pt_optim_dict['enpos']
      pred = pt_optim_dict['pred']
      for j in range(stpos, enpos):
        st_idx = j - stpos
        en_idx = enpos - 1 - j
        if priority == 'center':          
          pred_eval = st_idx * en_idx
        elif priority == 'top':
          pred_eval = (en_idx) ** 2
        elif priority == 'bottom':
          pred_eval = (st_idx) ** 2
        else:
          None
        if optim_dicts[qry].get(j):
          if optim_dicts[qry][j][-1] < pred_eval:
            optim_dicts[qry][j] = [pred[st_idx], pred_eval]
        else:
          optim_dicts[qry][j] = [pred[st_idx], pred_eval]
    optim_dicts_all.append(optim_dicts)
  return optim_dicts_all

In [ ]:
# Showing F1 Score and Accuracy in all and by qry
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def show_statistics_valid(all_predited_list, all_tgt_list, thres_list):
  best_all_predited_list = []
  best_results = []
  for i in range(17):
    by_thres_predicted = []
    f1_list, acc_list = [], []
    for j, thres in enumerate(thres_list):
      by_qry_predicted = []
      by_qry_tgt = []
      for doc_pred, doc_tgt in zip(all_predited_list, all_tgt_list):
        by_qry_predicted += doc_pred[j][i]
        by_qry_tgt += doc_tgt[i]
      f1 = f1_score(by_qry_predicted, by_qry_tgt)
      acc = accuracy_score(by_qry_predicted, by_qry_tgt)
      f1_list.append(f1)
      acc_list.append(acc)
    bs = np.argmax(f1_list)
    print(f'Goal{i+1}, threshold: {thres_list[bs]}, f1_score: {round(f1_list[bs], 3)}, accuracy: {round(acc_list[bs], 3)}')
    best_all_predited_list.append([doc_pred[bs][i] for doc_pred in all_predited_list])
    best_results.append((thres_list[bs], round(f1_list[bs], 3), round(acc_list[bs], 3)))
  
  best_all_predited_list_t = np.array(best_all_predited_list).T.tolist()
  single_predicted = []
  single_tgt = []
  for doc_pred, doc_tgt in zip(best_all_predited_list_t, all_tgt_list):
    for qry_pred, qry_tgt in zip(doc_pred, doc_tgt):
      single_predicted += qry_pred
      single_tgt += qry_tgt
  best_f1 = f1_score(single_predicted, single_tgt)
  best_acc = accuracy_score(single_predicted, single_tgt)
  print(f'Total Best f1_score:{round(best_f1, 3)}, accuracy:{round(best_acc, 3)}')
  return best_results

In [ ]:
import torch
from glob import glob
import numpy as np

def validate_execute(validation_list):
  best_results_list = []
  for step, align in validation_list:
    print(f'step:{step}, align:{align}')
    dataset = torch.load(glob(work_dir + '/dataset.pt')[0])
    optim_list_all = []
    for i in range(1, 18):
      pt_optim_dicts_all = torch.load(glob(work_dir + '/results/' + str(i) + '/valid.step' + str(step) + '.optim_dicts_all.pt')[0])
      optim_dicts_all = gen_optim_list(pt_optim_dicts_all, str(i), align)
      optim_list_by_qry = []
      for optim_dict in optim_dicts_all:
        optim_list_by_qry.append(torch.tensor([optim_dict[str(i)][j][0] for j in sorted(optim_dict[str(i)])]))
      optim_list_all.append(optim_list_by_qry)

    thres_range = range(0, 70, 1)
    src_pred_list = []
    for i in range(17):
      src_pred_list_by_thres = []
      for j in thres_range:
        src_pred = predict_goals(optim_list_all[i], threshold=(j/100))
        src_pred_list_by_thres.append(src_pred)
      src_pred_list.append(src_pred_list_by_thres)
    src_pred_list_t = np.array(src_pred_list).T.tolist()
    best_results = show_statistics_valid(src_pred_list_t, dataset['tgt_list_valid'], [i/100 for i in thres_range])
    best_results_list.append((step, align, best_results))
  return best_results_list

In [ ]:
validation_list = [(i, j) for j in ['center', 'top', 'bottom'] for i in range(2000, 10001, 2000)]
validation_list += [(i, j) for j in ['center', 'top', 'bottom'] for i in range(11000, 24001, 1000)]
best_results_list = validate_execute(validation_list)
torch.save(best_results_list, work_dir + '/best_results_list.pt')  

step:2000, align:center


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, threshold: 0.01, f1_score: 0.089, accuracy: 0.974
Goal2, threshold: 0.01, f1_score: 0.203, accuracy: 0.98
Goal3, threshold: 0.01, f1_score: 0.302, accuracy: 0.883
Goal4, threshold: 0.03, f1_score: 0.31, accuracy: 0.958
Goal5, threshold: 0.04, f1_score: 0.447, accuracy: 0.943
Goal6, threshold: 0.01, f1_score: 0.129, accuracy: 0.964
Goal7, threshold: 0.06, f1_score: 0.451, accuracy: 0.938
Goal8, threshold: 0.03, f1_score: 0.407, accuracy: 0.913
Goal9, threshold: 0.04, f1_score: 0.289, accuracy: 0.838
Goal10, threshold: 0.02, f1_score: 0.285, accuracy: 0.965
Goal11, threshold: 0.16, f1_score: 0.286, accuracy: 0.933
Goal12, threshold: 0.05, f1_score: 0.374, accuracy: 0.916
Goal13, threshold: 0.09, f1_score: 0.408, accuracy: 0.942
Goal14, threshold: 0.01, f1_score: 0.286, accuracy: 0.953
Goal15, threshold: 0.02, f1_score: 0.352, accuracy: 0.957
Goal16, threshold: 0.03, f1_score: 0.209, accuracy: 0.928
Goal17, threshold: 0.01, f1_score: 0.182, accuracy: 0.696


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Total Best f1_score:0.3, accuracy:0.923
step:4000, align:center


...

Total Best f1_score:0.293, accuracy:0.917
step:24000, align:bottom


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, threshold: 0.01, f1_score: 0.219, accuracy: 0.987
Goal2, threshold: 0.04, f1_score: 0.295, accuracy: 0.982
Goal3, threshold: 0.01, f1_score: 0.385, accuracy: 0.927
Goal4, threshold: 0.19, f1_score: 0.488, accuracy: 0.979
Goal5, threshold: 0.05, f1_score: 0.079, accuracy: 0.313
Goal6, threshold: 0.07, f1_score: 0.402, accuracy: 0.989
Goal7, threshold: 0.04, f1_score: 0.519, accuracy: 0.952
Goal8, threshold: 0.01, f1_score: 0.469, accuracy: 0.916
Goal9, threshold: 0.09, f1_score: 0.368, accuracy: 0.915
Goal10, threshold: 0.04, f1_score: 0.335, accuracy: 0.967
Goal11, threshold: 0.04, f1_score: 0.481, accuracy: 0.962
Goal12, threshold: 0.18, f1_score: 0.541, accuracy: 0.943
Goal13, threshold: 0.04, f1_score: 0.511, accuracy: 0.952
Goal14, threshold: 0.12, f1_score: 0.534, accuracy: 0.976
Goal15, threshold: 0.04, f1_score: 0.51, accuracy: 0.972
Goal16, threshold: 0.31, f1_score: 0.412, accuracy: 0.975
Goal17, threshold: 0.06, f1_score: 0.299, accuracy: 0.928


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Total Best f1_score:0.303, accuracy:0.92


In [ ]:
# Showing Optimal Hyperparameters (Model, Threshold, and Alignment)
import numpy as np
best_results_list = torch.load(glob(work_dir + '/best_results_list.pt')[0])
params_list = []
for i in range(17):
  best_result = best_results_list[np.argmax([b[i][1] for steps, align, b in best_results_list])]
  print('Goal' + str(i+1), best_result[0], best_result[1], best_result[2][i][0])
  params_list.append((i+1, best_result[0], best_result[1], best_result[2][i][0]))

Goal1 24000 bottom 0.01
Goal2 18000 top 0.12
Goal3 23000 center 0.02
Goal4 18000 center 0.18
Goal5 8000 center 0.07
Goal6 22000 center 0.07
Goal7 15000 center 0.22
Goal8 21000 center 0.08
Goal9 22000 center 0.01
Goal10 21000 center 0.03
Goal11 24000 center 0.01
Goal12 24000 center 0.1
Goal13 24000 center 0.05
Goal14 24000 center 0.02
Goal15 22000 center 0.05
Goal16 15000 center 0.17
Goal17 24000 center 0.02


In [ ]:
# Showing F1 Score and Accuracy in all an by qry
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def show_statistics_valid_test(all_predited_list, all_tgt_list):
  by_qry_tgt_list = []
  by_qry_pred_list = []
  for i in range(17):
    by_qry_tgt = []
    for data in all_tgt_list:
      by_qry_tgt += data[i]
    by_qry_pred = []
    for data in all_predited_list:
      by_qry_pred += data[i]
    f1 = f1_score(by_qry_pred, by_qry_tgt)
    acc = accuracy_score(by_qry_pred, by_qry_tgt)
    print(f'Goal{i+1}, f1_score: {round(f1, 3)}, accuracy: {round(acc, 3)}')

    by_qry_tgt_list.append(by_qry_tgt)
    by_qry_pred_list.append(by_qry_pred)
  
  tgt_list_all = []
  pred_list_all = []
  for by_qry_tgt in by_qry_tgt_list:
    tgt_list_all += by_qry_tgt
  for by_qry_pred in by_qry_pred_list:
    pred_list_all += by_qry_pred
  
  f1_all = f1_score(pred_list_all, tgt_list_all)
  acc_all = accuracy_score(pred_list_all, tgt_list_all)
  print(f'Total, f1_score: {round(f1_all, 3)}, accuracy: {round(acc_all, 3)}')

In [ ]:
import torch
from glob import glob
import numpy as np

def valid_test_execute(params_list, vt_flag):
  dataset = torch.load(glob(work_dir + '/dataset.pt')[0])
  optim_list_all = []
  for i, step, align, _ in params_list:
    pt_optim_dicts_all = torch.load(glob(work_dir + '/results/' + str(i) + '/' + vt_flag + '.step' + str(step) + '.optim_dicts_all.pt')[0])
    optim_dicts_all = gen_optim_list(pt_optim_dicts_all, str(i), align)
    optim_list_by_qry = []
    for optim_dict in optim_dicts_all:
      optim_list_by_qry.append(torch.tensor([optim_dict[str(i)][j][0] for j in sorted(optim_dict[str(i)])]))
    optim_list_all.append(optim_list_by_qry)

  src_pred_list = []
  data_type = 'tgt_list_' + vt_flag
  for i, _, _, threshold in params_list:
    src_pred = predict_goals(optim_list_all[i-1], threshold=threshold)
    src_pred_list.append(src_pred)
  src_pred_list_t = np.array(src_pred_list).T.tolist()
  show_statistics_valid_test(src_pred_list_t, dataset[data_type])

In [ ]:
valid_test_execute(params_list, 'valid')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, f1_score: 0.219, accuracy: 0.987
Goal2, f1_score: 0.507, accuracy: 0.991
Goal3, f1_score: 0.433, accuracy: 0.915
Goal4, f1_score: 0.549, accuracy: 0.982
Goal5, f1_score: 0.495, accuracy: 0.963
Goal6, f1_score: 0.427, accuracy: 0.99
Goal7, f1_score: 0.547, accuracy: 0.95
Goal8, f1_score: 0.505, accuracy: 0.919
Goal9, f1_score: 0.422, accuracy: 0.923
Goal10, f1_score: 0.373, accuracy: 0.976
Goal11, f1_score: 0.489, accuracy: 0.963
Goal12, f1_score: 0.586, accuracy: 0.946
Goal13, f1_score: 0.537, accuracy: 0.955
Goal14, f1_score: 0.557, accuracy: 0.977
Goal15, f1_score: 0.545, accuracy: 0.974
Goal16, f1_score: 0.439, accuracy: 0.976
Goal17, f1_score: 0.301, accuracy: 0.932
Total, f1_score: 0.48, accuracy: 0.96


In [ ]:
################ Test ################

In [ ]:
!cp $MODULE_PATH/bert_config_uncased_base_proposed.json ../bert_config_uncased_base.json

%env LOG_PATH=$work_dir/logs/proposed

for idx, steps, _, _ in params_list:
  %env RESULT_PATH=$work_dir/results/$idx/ 
  %env BERT_DATA_PATH=$work_dir/pt_data/$idx/
  %env MODEL_PATH=$work_dir/models/proposed$idx
  model_name = 'model_step_' + str(steps) + '.pt'
  %env MODEL_NAME=$model_name
  !python $MODULE_PATH/src/train.py -mode test -summarizer proposed -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/$MODEL_NAME -block_trigram true

env: LOG_PATH=/content/drive/MyDrive/work_proposed/logs/proposed
env: RESULT_PATH=/content/drive/MyDrive/work_proposed/results/1/
env: BERT_DATA_PATH=/content/drive/MyDrive/work_proposed/pt_data/1/
env: MODEL_PATH=/content/drive/MyDrive/work_proposed/models/proposed1
env: MODEL_NAME=model_step_24000.pt
[2022-04-24 05:57:21,512 INFO] Loading checkpoint from /content/drive/MyDrive/work_proposed/models/proposed1/model_step_24000.pt
Namespace(accum_count=1, batch_size=512, bert_config_path='../bert_config_uncased_base.json', bert_data_path='/content/drive/MyDrive/work_proposed/pt_data/1/', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, ff_size=2048, gpu_ranks=[0], heads=8, hidden_size=128, inter_layers=2, log_file='/content/drive/MyDrive/work_proposed/logs/proposed', lr=1, max_grad_norm=0, mode='test', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='/conten

In [ ]:
# Showing Test Results
valid_test_execute(params_list, 'test')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, f1_score: 0.156, accuracy: 0.983
Goal2, f1_score: 0.24, accuracy: 0.988
Goal3, f1_score: 0.299, accuracy: 0.901
Goal4, f1_score: 0.403, accuracy: 0.976
Goal5, f1_score: 0.362, accuracy: 0.955
Goal6, f1_score: 0.588, accuracy: 0.988
Goal7, f1_score: 0.467, accuracy: 0.95
Goal8, f1_score: 0.361, accuracy: 0.891
Goal9, f1_score: 0.409, accuracy: 0.915
Goal10, f1_score: 0.209, accuracy: 0.967
Goal11, f1_score: 0.317, accuracy: 0.94
Goal12, f1_score: 0.454, accuracy: 0.932
Goal13, f1_score: 0.523, accuracy: 0.941
Goal14, f1_score: 0.428, accuracy: 0.968
Goal15, f1_score: 0.467, accuracy: 0.974
Goal16, f1_score: 0.305, accuracy: 0.971
Goal17, f1_score: 0.205, accuracy: 0.93
Total, f1_score: 0.379, accuracy: 0.951
